# CSE 158, Fall 2019: Assignment 1 - Category

**Name:** CUI, HONGJIAN &ensp;&ensp;&ensp;&ensp; **PID:** U08398995 

In [1]:
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
# import nltk
# nltk.download('stopwords')

In [2]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [3]:
path = "/home/cui/Projects/PycharmProjects/CSE-158/data/train_Category.json"

In [4]:
dataset = list(parseData(path))

In [5]:
dataset[0]

{'n_votes': 0,
 'review_id': 'r99763621',
 'user_id': 'u17334941',
 'review_text': "Genuinely enthralling. If Collins or Bernard did invent this out of whole cloth, they deserve a medal for imagination. Lets leave the veracity aside for a moment - always a touchy subject when it comes to real life stories of the occult - and talk about the contents. \n The Black Alchemist covers a period of two years in which Collins, a magician, and Bernard, a psychic, undertook a series of psychic quests that put them in opposition with the titular Black Alchemist. As entertainment goes, the combination of harrowing discoveries, ancient lore, and going down the pub for a cigarette and a Guinness, trying to make sense of it all while a hen party screams at each other, is a winner. It is simultaneously down to earth and out of this world. \n It reads fast, both because of the curiousity and because Collins has a very clear writing style. Sometimes its a little clunky or over repetitive and there's a fe

In [6]:
split = 190000

In [7]:
training_set = dataset[:split]
validation_set = dataset[split:]

In [8]:
wordCounts = defaultdict(int)
punctuation = set(string.punctuation)
totalWords = 0
stemmer = PorterStemmer()
stopwords = stopwords.words("english")

In [9]:
for d in training_set:
    text = d['review_text']
    text = text.lower()
    text = [c for c in text if not (c in punctuation)]
    text = ''.join(text)
    words = text.strip().split()
    for word in words:
        w = stemmer.stem(word)
        if w in stopwords: continue
        totalWords += 1
        wordCounts[w] += 1

In [10]:
def feature(datum): 
    text = datum['review_text']
    text = text.lower()
    text = [c for c in text if not (c in punctuation)]
    text = ''.join(text)
    
    feat = np.zeros(len(wordSet), dtype="uint8")
    word = text.strip().split()
    for w in word:
        w = stemmer.stem(w)
        if not (w in wordSet): continue
        if w in stopwords: continue
        feat[wordId[w]] += 1
    
    return feat

In [11]:
popularWords = sorted(wordCounts.items(), key=lambda x:x[1])[-5000:]
popularWords.reverse()
wordSet = [w[0] for w in popularWords]
wordId = dict(zip(wordSet, range(len(wordSet))))

X_train = np.array([feature(d) for d in training_set])
X_validation = np.array([feature(d) for d in validation_set])
y_train = np.array([d['genreID'] for d in training_set], dtype="uint8")
y_validation = np.array([d['genreID'] for d in validation_set], dtype="uint8")

In [ ]:
model = LogisticRegression(solver='sag', C=10, multi_class='auto', max_iter=1000)
model.fit(X_train, y_train)

dataset_test = list(parseData("/home/cui/Projects/PycharmProjects/CSE-158/data/test_Category.json"))

predictions = open("/home/cui/Projects/PycharmProjects/CSE-158/data/predictions_Category_sag.txt", 'w')
predictions.write("userID-reviewID,prediction\n")

for l in dataset_test:
    cat = model.predict([feature(l)])
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(cat[0]) + "\n")

predictions.close()

In [ ]:
model = LogisticRegression(solver='lbfgs', C=10, multi_class='auto', max_iter=1000)
model.fit(X_train, y_train)
pred = model.predict(X_validation)
prediction = pred == y_validation
print(sum(prediction) / len(prediction))

dataset_test = list(parseData("/home/cui/Projects/PycharmProjects/CSE-158/data/test_Category.json"))

predictions = open("/home/cui/Projects/PycharmProjects/CSE-158/data/predictions_Category_lbfgs.txt", 'w')
predictions.write("userID-reviewID,prediction\n")

for l in dataset_test:
    cat = model.predict([feature(l)])
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(cat[0]) + "\n")

predictions.close()